In [53]:
import pickle
import sklearn as sk
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np

In [54]:
data_folder = 'C:\\Users\\EMBA\\Documents\\Technical_Training\\Machine Learning - Stock Market trading\\Yes Bank\\Data\\Featurized\\'
model_folder = 'C:\\Users\\EMBA\\Documents\\Technical_Training\\Machine Learning - Stock Market trading\\Yes Bank\\Model\\'
simulation_folder = 'C:\\Users\\EMBA\\Documents\\Technical_Training\\Machine Learning - Stock Market trading\\Yes Bank\\Data\\Simulation\\'
file_name = 'YESBANK.csv'
model ='Comprehensive_model.sav'
model_2 = 'Yesbank_model_OneVsRest_2017-10-26.sav'
data_file_path = data_folder +  file_name
model_file_path = model_folder + model
model_file_path_2 = model_folder + model_2
simulation_file_path = simulation_folder + file_name
stock_data = pd.read_csv(data_file_path)

In [55]:
stock_data = stock_data.replace([np.inf, -np.inf], np.nan)

In [56]:
stock_data.fillna(0,inplace=True)

In [57]:
attr_cols = stock_data.columns

In [58]:
feature_cols = []
static_cols = []
signal_cols = []
for i in attr_cols:
    attr_list=[]
    attr_list = i.split('_')
    if 'RBC' in attr_list or 'DBC' in attr_list or 'RBCV'in attr_list or 'DBCV' in attr_list or 'Add' in attr_list:
        feature_cols.append(i)
    if 'Signal' in attr_list:
        signal_cols.append(i)
features = stock_data[feature_cols]
feature_cols.append('Date')
feature_cols.append('Open')
feature_cols.append('Close')
feature_cols.append('Volume')
feature_cols.append('% Change')
#feature_cols.append('% V Change')
feature_cols

['Add_V_Spike',
 'Add_CTrend',
 'Add_OTrend',
 'DBC_Max_10',
 'DBC_Min_10',
 'DBC_Mean_10',
 'DBCV_Max_10',
 'DBCV_Min_10',
 'DBCV_Mean_10',
 'Add_RSI_10',
 'DBC_Max_15',
 'DBC_Min_15',
 'DBC_Mean_15',
 'DBCV_Max_15',
 'DBCV_Min_15',
 'DBCV_Mean_15',
 'Add_RSI_15',
 'DBC_Max_20',
 'DBC_Min_20',
 'DBC_Mean_20',
 'DBCV_Max_20',
 'DBCV_Min_20',
 'DBCV_Mean_20',
 'Add_RSI_20',
 'DBC_Max_25',
 'DBC_Min_25',
 'DBC_Mean_25',
 'DBCV_Max_25',
 'DBCV_Min_25',
 'DBCV_Mean_25',
 'Add_RSI_25',
 'DBC_Max_30',
 'DBC_Min_30',
 'DBC_Mean_30',
 'DBCV_Max_30',
 'DBCV_Min_30',
 'DBCV_Mean_30',
 'Add_RSI_30',
 'DBC_Max_50',
 'DBC_Min_50',
 'DBC_Mean_50',
 'DBCV_Max_50',
 'DBCV_Min_50',
 'DBCV_Mean_50',
 'Add_RSI_50',
 'DBC_Max_150',
 'DBC_Min_150',
 'DBC_Mean_150',
 'DBCV_Max_150',
 'DBCV_Min_150',
 'DBCV_Mean_150',
 'Add_RSI_150',
 'DBC_Max_200',
 'DBC_Min_200',
 'DBC_Mean_200',
 'DBCV_Max_200',
 'DBCV_Min_200',
 'DBCV_Mean_200',
 'Add_RSI_200',
 'Date',
 'Open',
 'Close',
 'Volume',
 '% Change']

In [59]:
stock_data = stock_data[feature_cols]
stock_data = stock_data[200:]
features = features[200:]

In [60]:
features.head()

,Add_V_Spike,Add_CTrend,Add_OTrend,DBC_Max_10,DBC_Min_10,DBC_Mean_10,DBCV_Max_10,DBCV_Min_10,DBCV_Mean_10,Add_RSI_10,...,DBCV_Min_150,DBCV_Mean_150,Add_RSI_150,DBC_Max_200,DBC_Min_200,DBC_Mean_200,DBCV_Max_200,DBCV_Min_200,DBCV_Mean_200,Add_RSI_200
200,0,1,1,0.000000,16.151755,-5.006997,356.791705,0.000000,128.874057,106.708014,...,2.696121,150.967277,240.754774,0.000000,87.161572,-29.495800,15584.004778,0.416457,503.594201,304.806110
201,0,0,0,2.096236,8.587688,-1.805620,811.618754,0.000000,306.760382,123.278477,...,0.636700,395.105294,259.947675,2.096236,83.318777,-27.797523,31200.552839,0.098348,1040.058551,297.673074
202,0,1,1,1.228153,6.596168,-1.771373,514.129127,5.313723,156.723718,133.060396,...,1.638611,230.269595,264.128634,1.228153,72.394137,-28.181861,20986.206402,0.253109,657.394856,284.081844
203,0,0,0,3.777240,3.977840,1.041164,224.916597,16.043408,48.791912,251.786280,...,2.888870,130.882126,291.371447,3.777240,68.159609,-26.172881,14884.295891,0.446230,402.082263,322.897269
204,0,0,0,6.563893,0.000000,3.878664,203.107219,18.257905,23.273520,619.386814,...,3.199741,113.957221,299.293688,6.563893,63.762223,-24.009203,13878.504965,0.494249,334.594773,356.115078


In [61]:
#features = stock_data[['% Change', '% V_Change', 
#        'Roll_Change_30', 'Roll_VChange_30', 'DB_CMax_30', 'DB_CMin_30', 'DB_CMean_30', 'DB_CVMax_30', 'DB_CVMin_30', 'DB_CVMean_30', 
#        'Roll_Change_50', 'Roll_VChange_50', 'DB_CMax_50', 'DB_CMin_50', 'DB_CMean_50', 'DB_CVMax_50', 'DB_CVMin_50', 'DB_CVMean_50', 
#       'Roll_Change_150', 'Roll_VChange_150', 'DB_CMax_150', 'DB_CMin_150', 'DB_CMean_150', 'DB_CVMax_150', 'DB_CVMin_150', 'DB_CVMean_150',
#       'Roll_Change_200', 'Roll_VChange_200', 'DB_CMax_200', 'DB_CMin_200', 'DB_CMean_200', 'DB_CVMax_200', 'DB_CVMin_200', 'DB_CVMean_200']]

In [62]:
#features = stock_data[['% Change',  
#        'Roll_Change_30', 'DB_CMax_30', 'DB_CMin_30', 'DB_CMean_30', 'DB_CVMean_30', 
#        'Roll_Change_50', 'DB_CMax_50', 'DB_CMin_50', 'DB_CMean_50',  'DB_CVMean_50',
#        'Roll_Change_150', 'DB_CMax_150', 'DB_CMin_150', 'DB_CMean_150', 'DB_CVMean_150',
#        'Roll_Change_200', 'DB_CMax_200', 'DB_CMin_200', 'DB_CMean_200', 'DB_CVMean_200']]

In [63]:
model =  pickle.load(open(model_file_path, 'rb'))

In [64]:
#simulation = pd.read_csv(simulation_file_path,dtype={'Stock':str,'Date':object,'Action':str, 'Price':float, 'Qty':int,'Value':float,'Cash_Balance':float,'Net_Qty':int})
#simulation = pd.read_csv(simulation_file_path)

In [65]:
features.iloc[0]

Add_V_Spike          0.000000
Add_CTrend           1.000000
Add_OTrend           1.000000
DBC_Max_10           0.000000
DBC_Min_10          16.151755
DBC_Mean_10         -5.006997
DBCV_Max_10        356.791705
DBCV_Min_10          0.000000
DBCV_Mean_10       128.874057
Add_RSI_10         106.708014
DBC_Max_15           0.000000
DBC_Min_15          16.151755
DBC_Mean_15         -6.735105
DBCV_Max_15        356.791705
DBCV_Min_15         14.299115
DBCV_Mean_15        83.493958
Add_RSI_15         200.760289
DBC_Max_20           0.000000
DBC_Min_20          16.151755
DBC_Mean_20         -7.400839
DBCV_Max_20        356.791705
DBCV_Min_20         14.299115
DBCV_Mean_20        67.300529
Add_RSI_20         308.137155
DBC_Max_25           0.000000
DBC_Min_25          16.151755
DBC_Mean_25         -7.038730
DBCV_Max_25        356.791705
DBCV_Min_25         14.299115
DBCV_Mean_25        56.766010
Add_RSI_25         346.640321
DBC_Max_30           0.000000
DBC_Min_30          16.467391
DBC_Mean_3

In [66]:
from sklearn.preprocessing import normalize
features = normalize(features)
features[0]

array([  0.00000000e+00,   6.32653450e-05,   6.32653450e-05,
         0.00000000e+00,   1.02184637e-03,  -3.16769405e-04,
         2.25725503e-02,   0.00000000e+00,   8.15326168e-03,
         6.75091929e-03,   0.00000000e+00,   1.02184637e-03,
        -4.26098741e-04,   2.25725503e-02,   9.04638425e-04,
         5.28227407e-03,   1.27011690e-02,   0.00000000e+00,
         1.02184637e-03,  -4.68216663e-04,   2.25725503e-02,
         9.04638425e-04,   4.25779116e-03,   1.94944034e-02,
         0.00000000e+00,   1.02184637e-03,  -4.45307706e-04,
         2.25725503e-02,   9.04638425e-04,   3.59132120e-03,
         2.19303195e-02,   0.00000000e+00,   1.04181519e-03,
        -4.58671291e-04,   2.25725503e-02,   9.04638425e-04,
         3.54263551e-03,   1.25057841e-02,   0.00000000e+00,
         2.12593594e-03,  -7.58770840e-04,   2.25725503e-02,
         9.04638425e-04,   3.10647777e-03,   1.20038302e-02,
         0.00000000e+00,   4.43359521e-03,  -1.69849099e-03,
         1.46942301e-01,

In [67]:
model = pickle.load(open(model_file_path, 'rb'))
model_OneVsRest = pickle.load(open(model_file_path_2, 'rb'))

In [68]:
first_row = {'Stock':['YesBank'],'Date':['2001-01-01'],'Action':['NA'],'Price':[0.0],'Qty':[0],'Value':[0.0],'Cash_Balance':[100000.0],'Net_Qty':[0]}

In [69]:
#new_row = {'Stock':[stock],'Date':[date],'Action':[action],'Price':[price],'Qty':[qty],'Value':[value],'Cash_Balance':[cash_balance],'Net_Qty':[net_qty]}

In [70]:
simulation = pd.DataFrame(data=first_row,index=None,columns=('Stock','Date','Action', 'Price', 'Qty','Value','Cash_Balance','Net_Qty'))
simulation_2 = pd.DataFrame(data=first_row,index=None,columns=('Stock','Date','Action', 'Price', 'Qty','Value','Cash_Balance','Net_Qty'))

In [71]:
#simulation = simulation.append(new_row,ignore_index=True)

In [72]:
features.shape

(2826, 59)

In [73]:
stock = 'YesBank'
transact = False

for i in range(len(features)):
    #X = features.iloc[i].reshape(-1,34)
    #X = features.iloc[i].reshape(-1,features.shape[1])
    X = features[i].reshape(-1,features.shape[1])
    #signal = model.predict(features.iloc[i])
    signal = model.predict(X)
    l = len(simulation)
    cash_balance = simulation.iloc[l-1]['Cash_Balance']
    
    net_qty = simulation.iloc[l-1]['Net_Qty']
    if signal!= 0:
        print ('cash balance ' +str(cash_balance))
        print ('net qty ' +str(net_qty))
    if i+1 > len(features)-1:
        price = stock_data.iloc[i]['Close']
        date = stock_data.iloc[i]['Date']
    else:
        price = stock_data.iloc[i+1]['Open']
        date = stock_data.iloc[i+1]['Date']
    if signal == 1:
        action = 'Buy'
        investment_fund = float(cash_balance) * 0.2
        if investment_fund < price:
            investment_fund = 0.0
        else:
            qty = int(investment_fund / price)
            value = float(qty * price)
            cash_balance = float(cash_balance - value)
            net_qty = net_qty + qty
            transact = True
    elif signal == -1:
        action = 'Sell'
        #if int(net_qty) > 0:
        if int(net_qty) <= 0:
            print ('net qty > 0')
            #qty = int(net_qty * 0.25)
            qty = 10
            #if qty < 1:
            #    qty = net_qty
            value = qty * price
            cash_balance = float(cash_balance + value)
            net_qty = net_qty - qty
            transact = True
    if transact == True:
        new_row = {'Stock':stock,'Date':date,'Action':action,'Price':price,
               'Qty':qty,'Value':value,'Cash_Balance':cash_balance,'Net_Qty':net_qty}
        simulation = simulation.append(new_row,ignore_index=True)
        transact = False

cash balance 100000.0
net qty 0
cash balance 80009.679022
net qty 978
cash balance 64024.079022
net qty 1754
cash balance 51233.279022
net qty 2381
cash balance 40992.479022
net qty 2883
cash balance 32795.819022
net qty 3270
cash balance 26251.379022
net qty 3588
cash balance 21010.979022
net qty 3852
cash balance 16829.179022
net qty 4055
cash balance 13480.379022
net qty 4237
cash balance 10790.459022
net qty 4381
cash balance 8642.858904
net qty 4499
cash balance 6924.218904
net qty 4592
cash balance 5556.938904
net qty 4664
cash balance 4455.938904
net qty 4724
cash balance 3582.338856
net qty 4772
cash balance 2882.338856
net qty 4800
cash balance 2323.438879
net qty 4823
cash balance 1869.298879
net qty 4841
cash balance 1518.458893
net qty 4855
cash balance 1240.048904
net qty 4866
cash balance 1021.168904
net qty 4872
cash balance 835.668914
net qty 4877
cash balance 687.468918
net qty 4881
cash balance 570.558915
net qty 4884
cash balance 493.758911
net qty 4886
cash balance 

In [74]:
stock = 'YesBank'
from sklearn.preprocessing import normalize
features = normalize(features)
transact = False

for i in range(len(features)):
    #X = features.iloc[i].reshape(-1,34)
    #X = features.iloc[i].reshape(-1,features.shape[1])
    X = features[i].reshape(-1,features.shape[1])
    #signal = model.predict(features.iloc[i])
    signal = model_OneVsRest.predict(X)
    l = len(simulation_2)
    cash_balance = simulation_2.iloc[l-1]['Cash_Balance']
    
    net_qty = simulation_2.iloc[l-1]['Net_Qty']
    if signal.sum() == 1 and signal[0][1] != 1:
        print ('cash balance ' +str(cash_balance))
        print ('net qty ' +str(net_qty))
        if i+1 > len(features)-1:
            price = stock_data.iloc[i]['Close']
            date = stock_data.iloc[i]['Date']
        else:
            price = stock_data.iloc[i+1]['Open']
            date = stock_data.iloc[i+1]['Date']
        if signal[0][2] == 1:
            print ('Buy')
            action = 'Buy'
            #investment_fund = float(cash_balance) * 0.2
            #investment_fund = float(cash_balance)
            investment_fund = float(100000.0)
            if investment_fund < price:
                investment_fund = 0.0
            else:
                qty = int(investment_fund / price)
                #qty = 1
                value = float(qty * price)
                #cash_balance = float(cash_balance - value)
                cash_balance = float(cash_balance + value)
                net_qty = net_qty + qty
                transact = True
        elif signal[0][0] == 1:
            print ('Sell')
            action = 'Sell'
            if int(net_qty) > 0:
                print ('net qty > 0')
                #qty = int(net_qty * 0.25)
                qty = int(net_qty * 1)
                #qty = 1
                if qty < 1:
                    qty = net_qty
                value = qty * price
                cash_balance = float(cash_balance + value)
                net_qty = net_qty - qty
                transact = True
        if transact == True:
            new_row = {'Stock':stock,'Date':date,'Action':action,'Price':price,
                   'Qty':qty,'Value':value,'Cash_Balance':cash_balance,'Net_Qty':net_qty}
            simulation_2 = simulation_2.append(new_row,ignore_index=True)
            transact = False

In [75]:
target_folder = 'C:\\Users\\EMBA\\Documents\\Technical_Training\\Machine Learning - Stock Market trading\\Yes Bank\\Data\\Simulation\\'
file_name1 = 'Yesbank_model_2017-10-26.csv'
file_name2 ='Yesbank_model_OneVsRest_2017-10-26.sav'
target_file_path_1 = target_folder + file_name1
target_file_path_2 = target_folder + file_name2
simulation.to_csv(target_file_path_1)
simulation_2.to_csv(target_file_path_2)

In [76]:
simulation_2

,Stock,Date,Action,Price,Qty,Value,Cash_Balance,Net_Qty
0,YesBank,2001-01-01,NA,0.0,0,0.0,100000.0,0


In [77]:
signal[0]

array([0, 0, 0])

In [78]:
signal[0][1]

0

In [79]:
simulation

,Stock,Date,Action,Price,Qty,Value,Cash_Balance,Net_Qty
0,YesBank,2001-01-01,NA,0.000000,0,0.000000,100000.000000,0
1,YesBank,5/11/2006,Buy,20.440001,978,19990.320978,80009.679022,978
2,YesBank,5/12/2006,Buy,20.600000,776,15985.600000,64024.079022,1754
3,YesBank,5/15/2006,Buy,20.400000,627,12790.800000,51233.279022,2381
4,YesBank,5/16/2006,Buy,20.400000,502,10240.800000,40992.479022,2883
5,YesBank,5/17/2006,Buy,21.180000,387,8196.660000,32795.819022,3270
6,YesBank,5/18/2006,Buy,20.580000,318,6544.440000,26251.379022,3588
7,YesBank,5/19/2006,Buy,19.850000,264,5240.400000,21010.979022,3852
8,YesBank,5/22/2006,Buy,20.600000,203,4181.800000,16829.179022,4055
9,YesBank,5/23/2006,Buy,18.400000,182,3348.800000,13480.379022,4237
